In [ ]:
import pandas as pd
from scipy.io import loadmat
import altair as alt
import numpy as np
import math
from scipy.interpolate import InterpolatedUnivariateSpline
from src.spinorama.load import graph_melt, parse_graph_freq_princeton_mat
from src.spinorama.graph import graph_freq, graph_params_default
from src.spinorama.analysis import compute_cea2034


alt.data_transformers.disable_max_rows()

In [ ]:
#h_mat = loadmat("datas/Princeton/Genelec 8351A/Genelec8351A_H_IR.mat")
#v_mat = loadmat("datas/Princeton/Genelec 8351A/Genelec8351A_V_IR.mat")
# h_mat = loadmat("datas/Princeton/Genelec 8030A/Genelec_H_IR.mat")
# v_mat = loadmat("datas/Princeton/Genelec 8030A/Genelec_V_IR.mat")
#h_mat = loadmat("datas/Princeton/KEF LS50/KEF50_H_IR.mat")
#v_mat = loadmat("datas/Princeton/KEF LS50/KEF50_V_IR.mat")
h_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_H_IR.mat")
v_mat = loadmat("datas/Princeton/Avantgarde Acoustic Solo/Avantgarde_V_IR.mat")

h_spl = parse_graph_freq_princeton_mat(h_mat, 'H')
v_spl = parse_graph_freq_princeton_mat(v_mat, 'V')

In [ ]:
v_spl

In [ ]:
cea2034_unmelted = compute_cea2034(h_spl, v_spl)
cea2034 = graph_melt(cea2034_unmelted)

params = graph_params_default
params['xmin'] = 500
chart_cea2034 = graph_freq(cea2034, graph_params=params)

chart_cea2034


In [ ]:
chart_early_reflections = alt.Chart(early_reflections).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,105])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)

chart_vertical_reflections = alt.Chart(vertical_reflections).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,105])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)

chart_horizontal_reflections = alt.Chart(horizontal_reflections).transform_filter(
    'datum.Freq>500 && datum.Freq<20000'
).mark_line(
).encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log")), 
    alt.Y('dB:Q', scale=alt.Scale(domain=[60,95])),
    alt.Color('Measurements:N')
).properties(
    width=900,
    height=400
)


chart_cea2034 
# chart_early_reflections
# chart_vertical_reflections
# chart_horizontal_reflections


In [ ]:
v_mat